In [1]:
from load_data import load_data
from preprocess import preprocess
from gensim.models import Word2Vec
from baselines import NaiveBayes

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacobleooskar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacobleooskar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jacobleooskar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/jacobleooskar/Documents/Education/ETHZ/Curriculum/Semester02/02CILab/Project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
train_path_neg = "../twitter-datasets/train_neg_full.txt"
train_path_pos = "../twitter-datasets/train_pos_full.txt"
test_path = "../twitter-datasets/test_data.txt"

In [3]:
X_train, y_train, X_val, y_val, X_test = load_data(train_path_neg, train_path_pos, test_path)

In [4]:
X_train_tokens, X_val_tokens, X_test_tokens = preprocess(X_train, X_val, X_test)

In [5]:
X_train_tokens.head()

0    [got, interview, next, thursday, best, thing, ...
1                  [lunch, bosco, stick, meatbal, sub]
2    [soo, good, look, miss, ton, ton, seeyousoon, ...
3    [wive, mariah, carey, beyonc, kati, perri, ple...
4                              [sleep, th, today, smh]
Name: tweet, dtype: object

In [6]:
embedder = Word2Vec(X_train_tokens, window=5, min_count=1, workers=4)

In [7]:
baseline_model = NaiveBayes()
baseline_model.fit(X_train_tokens, y_train)
baseline_model.evaluate(X_val_tokens, y_val)

[[82566 29934]
 [14964 97536]]
              precision    recall  f1-score   support

           0       0.85      0.73      0.79    112500
           1       0.77      0.87      0.81    112500

    accuracy                           0.80    225000
   macro avg       0.81      0.80      0.80    225000
weighted avg       0.81      0.80      0.80    225000

0.8004533333333334
